# Notebook use to check the result of the classifier, how well can you detect the nucleus .

You can click `shift` + `enter` to run one cell, you can also click run in top menu.
To run all the cells, you can click `kernel` and `Restart and run all` in the top menu.

In [ ]:
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 8,8
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [ ]:
import numpy as np
import javabridge
import bioformats
from itkwidgets import view
from sklearn.externals import joblib

In [ ]:
# Ignore warnings in notebook
import warnings
warnings.filterwarnings('ignore')

### The following path should direct to the folder "utils", on Window env it should have slash " / " and not backslash " \ " . 

In [ ]:
# Create a temporary python PATH to the module that we are using for the analysis
import sys
sys.path.insert(0, "/Users/Espenel/Desktop/Mini-Grant-Image-analysis/2018/Chloe/ChromosomeDetectionChloe/utils")

In [ ]:
from chromosome_dsb import *

# Loading a typical image using bioformats

In [ ]:
javabridge.start_vm(class_path=bioformats.JARS)

### In the path variable you should enter the path to your image of interest:

In [ ]:
path = '/Users/Espenel/Desktop/Mini-Grant-Image-analysis/2018/Chloe/data_chloe/cku-exo1_002/2017-04-12_RAD51-HTP3_cku80-exo1_002_visit_13_D3D_ALX.dv'

## in the following cell in "channel" enter the the channel (starting from 0) where you will find the nucleus

In [ ]:
img = load_data.load_bioformats(path, channel = 3, no_meta_direct = True)

In [ ]:
img.shape

In [ ]:
#view(visualization.convert_view(img))

# Sliding Window

### First need to load the classifier (clf) and scaler.

In [ ]:
clf = joblib.load("/Users/Espenel/Desktop/Mini-Grant-Image-analysis/2018/Chloe/clf_scaler/clf")
scaler = joblib.load("/Users/Espenel/Desktop/Mini-Grant-Image-analysis/2018/Chloe/clf_scaler/scaler")

In [ ]:
import time

In [ ]:
tp1 = time.time()
result = search.rolling_window(img, clf, scaler)
tp2 = time.time()

In [ ]:
print("It took {}sec to find the chromosomes in 1 Zstack".format(int(tp2-tp1)))

### Optionally you can create a Heat map with the probability at every pixel that there is a nucleus

In [ ]:
#heat_map = visualization.heatmap(result)

In [ ]:
#view(visualization.convert_view(heat_map))

### Max projection and check how the result looks like

In [ ]:
proj = np.amax(img, axis=0)

### When boxes are overlapping, only keep the highest probability one.
Here you can adjust `probaThresh` and `overlaThresh`, if you find better parameters, you can change them in the function `batch.batch` in the `chromosome_dsb` folder.

In [ ]:
box = search.non_max_suppression(result, probaThresh=0.8, overlapThresh=0.3)

In [ ]:
import matplotlib.patches as patches
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(proj, vmax = 100000)
for rec in box: 
    rect = patches.Rectangle((rec[0],rec[1]),70,70,linewidth=3,edgecolor='y',facecolor='none')
    # Add the patch to the Axes
    ax.add_patch(rect)
plt.axis('off')
#plt.savefig('/Users/Espenel/Desktop/Mini-Grant-Image-analysis/2018/Chloe/data_chloe/fig.png', bbox_inches="tight", pad_inches=0)

# Save the result

In [ ]:
#path = "/Users/Espenel/Desktop/Mini-Grant-Image-analysis/2018/Chloe/13/"

In [ ]:
#load_data.save_file(path, "bbox_3D", box, model=False)
#load_data.save_file(path, "bbox_3D", binary, model=False)